In [1]:
import numpy as np
import imageio
%matplotlib inline
import matplotlib.pyplot as plt
from utils import * # tic toc imshow

import higra as hg

In [2]:
image = imageio.imread("data/101087.jpg")
# switch to float to avoir numerical issue with uint8
image = image.astype(np.float64)/255 
# discard the color channel
size = image.shape[:2]
print("Image size:", size)
imshow(image)

Image size: (481, 321)


(<Figure size 481.5x721.5 with 1 Axes>,
 <matplotlib.axes._axes.Axes at 0x10bcd24f390>)

In [3]:
graph = hg.get_4_adjacency_graph(size)
edge_weights = hg.weight_graph(image, hg.WeightFunction.L2, graph)
tree, altitudes = hg.bpt_canonical(edge_weights)
saliency = hg.saliency(altitudes)
imshow(hg.graph_4_adjacency_2_khalimsky(saliency))

(<Figure size 961.5x1441.5 with 1 Axes>,
 <matplotlib.axes._axes.Axes at 0x10bcd25c470>)

In [4]:
def area_filter_bpt(saliency, area_threshold=50):
    # binary partition tree by altitude ordering (bpt)
    tree, altitudes = hg.bpt_canonical(saliency)
    # get mst as a byproduct of previous function
    mst =  hg.CptBinaryHierarchy.get_mst(tree)
    # compute area of each node
    area = hg.attribute_area(tree)      
    # compute for each node the minimum of the area of its children
    min_area_children = hg.accumulate_parallel(area, hg.Accumulators.min)    
    # identify nodes having a child whose area is too small
    deleted_frontier_nodes = min_area_children <= area_threshold  
    # the edge weights of the mst are the altitudes of the inner nodes of the bpt
    mst_edge_weights = altitudes[tree.num_leaves():]                    
    # set the edge weights of mst edges corresponding to identified nodes to 0
    mst_edge_weights[deleted_frontier_nodes[tree.num_leaves():]] = 0         
    # compute the bpt of the mst with the modified edge weights
    tree, altitudes = hg.bpt_canonical(mst_edge_weights, mst) 
    # return the saliency map corresponding to this last tree
    return hg.saliency(altitudes)                                             

In [5]:
filtersm = area_filter_bpt(saliency)

In [6]:
imshow(hg.graph_4_adjacency_2_khalimsky(filtersm))

(<Figure size 961.5x1441.5 with 1 Axes>,
 <matplotlib.axes._axes.Axes at 0x10bcd173d30>)

In [7]:
def area_filter_qfz(saliency, area_threshold=50):
    # compute the minimum spanning tree 
    mst = hg.minimum_spanning_tree(saliency)        
    # get the correspondance between mst edges and the edges of the input graph
    mst_edge_map = hg.CptMinimumSpanningTree.get_edge_map(mst)            
    # deduce mst edge weights from the correspondance
    mst_edge_weights = saliency[mst_edge_map]                       
    # compute qfz on the weighted mst
    tree, altitudes = hg.quasi_flat_zones_hierarchy(mst_edge_weights, mst)     
    
    # get all the edges of the mst as two lists containing respectively the sources and the targets vertices
    sources, targets = mst.edge_list()                 
    # compute the largest region containing a source vertex with an altitude lower than the weight of its edge
    region_sources = tree.find_region(sources, mst_edge_weights, altitudes)    
    # compute the largest region containing a target vertex with an altitude lower than the weight of its edge
    region_targets = tree.find_region(targets, mst_edge_weights, altitudes)   
    
    # stack the regions of sources and targets to reform edges
    region_edges = np.vstack((region_sources, region_targets)) 
    # area of each node of the tree
    area = hg.attribute_area(tree)  
    # area of the found regions            
    area_nodes = area[region_edges] 
    # minimum of the area of the regions founds at the two extremities on an edge
    min_area_children = np.min(area_nodes,  axis=0)  
    
    # identify nodes having a child whose area is too small
    deleted_frontier_nodes = min_area_children <= area_threshold 
    # set the edge weights of mst edges corresponding to identified nodes to 0
    mst_edge_weights[deleted_frontier_nodes] = 0
    # compute the bpt of the mst with the modified edge weights
    tree, altitudes = hg.bpt_canonical(mst_edge_weights, mst)
    # return the saliency map corresponding to this last tree
    return hg.saliency(altitudes)                                              

In [8]:
filtersm2 = area_filter_qfz(saliency)

In [9]:
imshow(hg.graph_4_adjacency_2_khalimsky(filtersm2))

(<Figure size 961.5x1441.5 with 1 Axes>,
 <matplotlib.axes._axes.Axes at 0x10bcd858048>)

In [10]:
imshow(hg.graph_4_adjacency_2_khalimsky(filtersm2)-hg.graph_4_adjacency_2_khalimsky(filtersm))

(<Figure size 961.5x1441.5 with 1 Axes>,
 <matplotlib.axes._axes.Axes at 0x10bcfd5bac8>)

In [11]:
%timeit filtersm = area_filter_bpt(saliency)

923 ms ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit filtersm2 = area_filter_qfz(saliency)

1.04 s ± 48.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%load_ext line_profiler

In [14]:
%lprun -f area_filter_bpt area_filter_bpt(saliency)


Timer unit: 3.94757e-07 s

Total time: 0.852804 s
File: <ipython-input-4-7d5cd09ee0fa>
Function: area_filter_bpt at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def area_filter_bpt(saliency, area_threshold=50):
     2                                               # binary partition tree by altitude ordering (bpt)
     3         1     726618.0 726618.0     33.6      tree, altitudes = hg.bpt_canonical(saliency)
     4                                               # get mst as a byproduct of previous function
     5         1         63.0     63.0      0.0      mst =  hg.CptBinaryHierarchy.get_mst(tree)
     6                                               # compute area of each node
     7         1      16996.0  16996.0      0.8      area = hg.attribute_area(tree)      
     8                                               # compute for each node the minimum of the area of its children
     9         1      21785.0

In [15]:
%lprun -f area_filter_qfz area_filter_qfz(saliency)

Timer unit: 3.94757e-07 s

Total time: 1.0023 s
File: <ipython-input-7-09e1663e036d>
Function: area_filter_qfz at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def area_filter_qfz(saliency, area_threshold=50):
     2                                               # compute the minimum spanning tree 
     3         1     333927.0 333927.0     13.2      mst = hg.minimum_spanning_tree(saliency)        
     4                                               # get the correspondance between mst edges and the edges of the input graph
     5         1         23.0     23.0      0.0      mst_edge_map = hg.CptMinimumSpanningTree.get_edge_map(mst)            
     6                                               # deduce mst edge weights from the correspondance
     7         1       3750.0   3750.0      0.1      mst_edge_weights = saliency[mst_edge_map]                       
     8                                            